In [1]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
env = gym.make('CartPole-v0')

[2017-02-24 15:55:55,757] Making new env: CartPole-v0


In [3]:
# Input and output size based on the Env
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
learning_rate = 0.1
print input_size, output_size

4 2


In [4]:
# Theses lines establish the feed-forward part of the network used to choose actions
# state input
X = tf.placeholder(tf.float32, [None,input_size], name="input_x") # state input
# weight
W = tf.get_variable("W", shape=[input_size, output_size], 
                    initializer=tf.contrib.layers.xavier_initializer())

Qpred = tf.matmul(X,W) # out Q prediction
Y = tf.placeholder(shape=[None,output_size], dtype=tf.float32) # Y label

loss = tf.reduce_sum(tf.square(Y - Qpred))

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [5]:
# Set Q-learning related parameters
dis = 0.99
num_episodes = 200

In [6]:
# Create lists to contrain total rewards and steps per episode
rList = []

In [7]:
# Initializing the variables
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
# Q-network training
for i in range(num_episodes):
    # Reset environment and get first new observation
    s = env.reset()
    e = 1. / ((i / 50) + 10)
    step_count = 0
    done = False
    
    # The Q-Network training
    while not done:
        step_count += 1
        x = np.reshape(s, [1, input_size])
        # Choose an action by greedily (with e chance of random action) from the Q-network
        Qs = sess.run(Qpred, feed_dict={X:x})
        if np.random.rand(1) < e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs)
            
        # Get new state and reward from environment
        s1, reward, done, _ = env.step(a)
        if done:
            # Update Q, and no Qs+1, sice it's a terminal state
            Qs[0, a] = -100
        else:
            x1 = np.reshape(s1, [1, input_size])
            # Obtain the Q_s1 values by feeding the new state through our network 
            Qs1 = sess.run(Qpred, feed_dict={X: x1})
            # update Q
            Qs[0, a] = reward + dis * np.max(Qs1)
        
        # Train our network using using target and predicted Q values on each episode    
        sess.run(train, feed_dict={X: x, Y:Qs})
        s = s1
    rList.append(step_count)
    print("Episode:{} steps:{}".format(i, step_count))
    if len(rList) > 10 and np.mean(rList[-10:]) > 200:
        break


            
            

Episode:0 steps:10
Episode:1 steps:9
Episode:2 steps:65
Episode:3 steps:42
Episode:4 steps:24
Episode:5 steps:24
Episode:6 steps:38
Episode:7 steps:15
Episode:8 steps:10
Episode:9 steps:10
Episode:10 steps:9
Episode:11 steps:9
Episode:12 steps:12
Episode:13 steps:34
Episode:14 steps:18
Episode:15 steps:26
Episode:16 steps:38
Episode:17 steps:23
Episode:18 steps:27
Episode:19 steps:24
Episode:20 steps:29
Episode:21 steps:31
Episode:22 steps:62
Episode:23 steps:24
Episode:24 steps:32
Episode:25 steps:10
Episode:26 steps:12
Episode:27 steps:16
Episode:28 steps:35
Episode:29 steps:25
Episode:30 steps:21
Episode:31 steps:18
Episode:32 steps:15
Episode:33 steps:43
Episode:34 steps:15
Episode:35 steps:31
Episode:36 steps:25
Episode:37 steps:51
Episode:38 steps:45
Episode:39 steps:29
Episode:40 steps:22
Episode:41 steps:55
Episode:42 steps:19
Episode:43 steps:11
Episode:44 steps:10
Episode:45 steps:9
Episode:46 steps:10
Episode:47 steps:47
Episode:48 steps:23
Episode:49 steps:67
Episode:50 ste

In [9]:
# See our trained network in action
observation = env.reset()
reward_sum = 0
while True:
    #env.render()
    x = np.reshape(observation, [1, input_size])
    Qs = sess.run(Qpred, feed_dict={X:x})
    a = np.argmax(Qs)
    
    observation, reward, done, _ = env.step(a)
    reward_sum += reward
    if done:
        print("Total score:{}".format(reward_sum))
        break

Total score:28.0
